In [0]:
df1 = spark.read.parquet("abfss://labdata@moviehistoryus.dfs.core.windows.net/invoices/invoices_101_200.parquet")
df2 = spark.read.parquet("abfss://labdata@moviehistoryus.dfs.core.windows.net/invoices/invoices_1_100.parquet")
df3 = spark.read.parquet("abfss://labdata@moviehistoryus.dfs.core.windows.net/invoices/invoices_201_99457.parquet")

In [0]:
#fusiona los tres datafreames en un nuevo dataframe 'df_union' que selecciona solo las columnas definidas
df_union = df1.union(df2).union(df3)
df_union = df_union.select("customer_id", "category", "price", "quantity", "invoice_date")

In [0]:
#escribe el dataframe en la tabla 'lc_ex1' particionada por la columna 'invoice_date'
df_union.write.mode("overwrite").partitionBy("invoice_date").saveAsTable("deltacatalog.deltadb.lc_ex1")

In [0]:
%sql
--ejecuta una optimización con Z-ORDER sobre la tabla Delta lc_ex1, lo que mejora significativamente el rendimiento de consultas filtradas por customer_id
--Agrupa valores similares de customer_id en los mismos archivos, lo que permite que Spark lea menos archivos cuando se filtra por ese campo.
OPTIMIZE deltacatalog.deltadb.lc_ex1
ZORDER BY customer_id; 

In [0]:
%sql
SELECT count(*)
FROM FROM deltacatalog.deltadb.lc_ex1; 

FROM
99457


In [0]:
#escribe el DataFrame 'df_union' como una tabla Delta optimizada por clustering físico, lo que puede mejorar el rendimiento de lectura en columnas no particionadas.
df_union.write.mode("overwrite").clusterBy("invoice_date", "customer_id").saveAsTable("deltacatalog.deltadb.lc_ex2")

#clusterBy("invoice_date", "customer_id"): organiza los datos físicamente en archivos que agrupan valores similares de esas columnas
#A diferencia de partitionBy, no crea carpetas físicas por valor, pero sí mejora el pruning de archivos cuando se filtra por esas columnas
#Es útil cuando las columnas tienen alta cardinalidad, no se requiere crear miles de carpetas físicas y se busca rendimiento sin overhead de particionado.

In [0]:
%sql
SELECT count(*)
FROM FROM deltacatalog.deltadb.lc_ex2; 

FROM
99457


In [0]:
%%time
spark.sql(
    """
    SELECT category,
        SUM(price * quantity) AS total_sales
    from deltacatalog.deltadb.lc_ex1
    WHERE (invoice_date BETWEEN '2021-01-01' AND '2023-12-31') AND 
        customer_id = 201
    GROUP BY category
    """
)

CPU times: user 2.03 ms, sys: 268 µs, total: 2.3 ms
Wall time: 145 ms


DataFrame[category: string, total_sales: double]

In [0]:
%%time
spark.sql(
    """
    SELECT category,
        SUM(price * quantity) AS total_sales
    from deltacatalog.deltadb.lc_ex2
    WHERE (invoice_date BETWEEN '2021-01-01' AND '2023-12-31') AND 
        customer_id = 201
    GROUP BY category
    """
)

CPU times: user 3.19 ms, sys: 509 µs, total: 3.7 ms
Wall time: 152 ms


DataFrame[category: string, total_sales: double]